* 方法参考自 The Barra China Equity Model (CNE5)'s 文档

* 请在环境变量中设置`DB_URI`指向数据库

In [1]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from alphamind.api import *
from PyFin.api import *
from alphamind.analysis.crosssetctions import cross_section_analysis

plt.style.use('ggplot')

In [2]:
"""
Back test parameter settings
"""

start_date = '2020-01-01'
end_date = '2020-02-21'
category = 'sw'
level = 1
freq = '10b'
universe = Universe('hs300')

horizon = map_freq(freq)
ref_dates = makeSchedule(start_date, end_date, freq, 'china.sse')

In [3]:
def risk_factor_analysis(factor_name):
    data_source = os.environ['DB_URI']
    engine = SqlEngine(data_source)
    risk_names = list(set(risk_styles).difference({factor_name}))
    industry_names = list(set(industry_styles).difference({factor_name}))
    constraint_risk = risk_names + industry_names
    
    df = pd.DataFrame(columns=['ret', 'ic', 't.'], dtype=float)

    for ref_date in ref_dates:
        df.loc[ref_date, :] = cross_section_analysis(ref_date,
                                                     factor_name,
                                                     universe,
                                                     horizon,
                                                     constraint_risk,
                                                     engine=engine)
    df.index = pd.to_datetime(df.index)
    return df

In [4]:
candidates_factors = ['BETA', 'SIZE']

In [5]:
%%time
res = [risk_factor_analysis(factor) for factor in candidates_factors]

2021-07-03 11:34:39,965 - ALPHA_MIND - WARNING - winsorize_normal normally should not be done after neutralize
2021-07-03 11:34:39,967 - ALPHA_MIND - INFO - 2020-01-02 00:00:00 is finished with 300 stocks for BETA
2021-07-03 11:34:39,968 - ALPHA_MIND - INFO - 2020-01-02 00:00:00 risk_exposure: 3.207509232803705e-31
2021-07-03 11:34:40,957 - ALPHA_MIND - WARNING - winsorize_normal normally should not be done after neutralize
2021-07-03 11:34:40,959 - ALPHA_MIND - INFO - 2020-01-16 00:00:00 is finished with 300 stocks for BETA
2021-07-03 11:34:40,960 - ALPHA_MIND - INFO - 2020-01-16 00:00:00 risk_exposure: 6.420490585482854e-31
2021-07-03 11:34:41,729 - ALPHA_MIND - WARNING - winsorize_normal normally should not be done after neutralize
2021-07-03 11:34:41,731 - ALPHA_MIND - INFO - 2020-02-07 00:00:00 is finished with 300 stocks for BETA
2021-07-03 11:34:41,733 - ALPHA_MIND - INFO - 2020-02-07 00:00:00 risk_exposure: 4.227519029223244e-31
2021-07-03 11:34:42,345 - ALPHA_MIND - WARNING - 

CPU times: user 5.84 s, sys: 182 ms, total: 6.02 s
Wall time: 7.34 s


In [6]:
df = pd.DataFrame()

for f_name, data in zip(candidates_factors, res):
    data['factor'] = f_name
    df = df.append(data)

In [7]:
df['abs t.'] = np.abs(df['t.'])
df[['factor', 'abs t.']].groupby('factor').mean().sort_values('abs t.', ascending=False).head()

abs t.
factor          
BETA    1.283202
SIZE    0.941047